## Using Unsloth to finetune

## Install Prerequisite Packages

In [ ]:
# This is necessary for colab
!pip install python-dotenv
!pip install datasets
!pip install plotly
!pip install nbformat
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-0hbirobv/unsloth_7ece29f83129443888b84ec24b54f40b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-0hbirobv/unsloth_7ece29f83129443888b84ec24b54f40b
  Resolved https://github.com/unslothai/unsloth.git to commit a2f8db3e7341f983af5814a2c56f54fa29ee548d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.10.7-py3-none-any.whl size=164377 sha256=c8373ee7d1e549f3d79ee9a9ed4aed159bd7b745ec8366ef8f6c71758fc31eb6
  Stored in directory: /tmp/pip-ephem-wheel-cache-med30h0b/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2024.10.6
    Uninstalling unsloth-2024.10.6:
      Successfully uninstalled unsloth-2024.10.6


## Load `.env`

In [1]:
import os
import sys

from datasets import Dataset

from dotenv import find_dotenv, load_dotenv

load_dotenv()

/home/owen/anaconda3/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Important Global Parameters

In [11]:
# FINETUNING_DATASET_NAME="CPSC532/2024NOV2_arxiv_qa_data_cleaned"
FINETUNING_DATASET_NAME="CPSC532/2024NOV6_qwen_2_5_model"
OUTPUT_MODEL_NAME="2024NOV6_qwen_2_5_model"
BASE_MODEL_NAME="unsloth/Llama-3.2-3B-Instruct"

## API Keys

In [3]:
# Could also insert the token here directly
HF_TOKEN = os.getenv("HUGGINGFACE_API_KEY")

Leveraging Unsloth notebooks for finetuning

In [4]:
max_seq_length = 16000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


In [5]:
from unsloth import FastLanguageModel
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_NAME, # or choose "unsloth/Llama-3.2-1B-Instruct"
    # model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3080 Ti. Max memory: 11.753 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset


## Get dataset

In [23]:
dataset_finetune = load_dataset(
    "CPSC532/arxiv_qa_data",
    "2024NOV6_qwen_2_5_model",
    split="train",
    token=HF_TOKEN
)

In [24]:
dataset_finetune

Dataset({
    features: ['filename', 'source', 'source_type', 'chunk', 'question', 'answer'],
    num_rows: 1068
})

In [25]:

dataset_finetune['question'][0]

'Where are the CLS tokens added to the augmented input tensor x[+]1:T +1, and what is their role before and after processing through the Transformer?'

In [26]:

dataset_finetune['answer'][0]

'The CLS tokens are added as an additional component (the \\((T+1)\\)-st component) to the augmented input tensor \\(x^{+}_{1:T+1}\\), which has a shape of \\([T + 1] \\times 2b\\). Specifically, the augmented input tensor is formed by combining the original input tensor \\(x_{1:T} \\in \\mathbb{R}[T \\times 2b]\\) with the CLS tokens:\n\n\\[ x^{+}_{1:T+1} = [x_1, \\ldots, x_T, c]^\\top \\]\n\nwhere \\(c\\) represents the CLS token.\n\n### Role Before Processing Through the Transformer\n\nBefore processing through the Transformer, the CLS tokens do not carry any covariate-specific information. They are initialized as part of the augmented input tensor but have not yet interacted with the covariates or undergone any transformation by the attention mechanism (2.4)-(2.6). Their role at this stage is to serve as a global prior parameter, providing initial context before being influenced by the covariate data.\n\n### Role After Processing Through the Transformer\n\nAfter processing through 

## Convert dataset to messages format

In [27]:
def convert_to_messages_format(example):
    return [
        {"role": "user", "content": example['question']},
        {"role": "assistant", "content": example['answer']},
    ]

In [28]:
dataset_finetune = dataset_finetune.map(
    lambda x: {
        'conversations' : convert_to_messages_format(x)
        }
)

Map: 100%|██████████| 1068/1068 [00:00<00:00, 15076.20 examples/s]


In [29]:
dataset_finetune = dataset_finetune.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 1068/1068 [00:00<00:00, 18775.59 examples/s]


In [30]:
dataset_finetune['text'][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhere are the CLS tokens added to the augmented input tensor x[+]1:T +1, and what is their role before and after processing through the Transformer?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe CLS tokens are added as an additional component (the \\((T+1)\\)-st component) to the augmented input tensor \\(x^{+}_{1:T+1}\\), which has a shape of \\([T + 1] \\times 2b\\). Specifically, the augmented input tensor is formed by combining the original input tensor \\(x_{1:T} \\in \\mathbb{R}[T \\times 2b]\\) with the CLS tokens:\n\n\\[ x^{+}_{1:T+1} = [x_1, \\ldots, x_T, c]^\\top \\]\n\nwhere \\(c\\) represents the CLS token.\n\n### Role Before Processing Through the Transformer\n\nBefore processing through the Transformer, the CLS tokens do not carry any covariate-specific information. They are

## Set Training Parameters

In [31]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_finetune,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 1,  # Affects memory usage
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1, # Affects memory usage
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 10, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
    ),
)

Map: 100%|██████████| 1068/1068 [00:00<00:00, 6513.37 examples/s]


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. Look into this

In [32]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map: 100%|██████████| 1068/1068 [00:00<00:00, 5527.58 examples/s]


In [33]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhere are the CLS tokens added to the augmented input tensor x[+]1:T +1, and what is their role before and after processing through the Transformer?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe CLS tokens are added as an additional component (the \\((T+1)\\)-st component) to the augmented input tensor \\(x^{+}_{1:T+1}\\), which has a shape of \\([T + 1] \\times 2b\\). Specifically, the augmented input tensor is formed by combining the original input tensor \\(x_{1:T} \\in \\mathbb{R}[T \\times 2b]\\) with the CLS tokens:\n\n\\[ x^{+}_{1:T+1} = [x_1, \\ldots, x_T, c]^\\top \\]\n\nwhere \\(c\\) represents the CLS token.\n\n### Role Before Processing Through the Transformer\n\nBefore processing through the Transformer, the CLS tokens do not carry any covariate-specific information. They are

In [34]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                      \n\n### What are CLS Tokens and How Are They Used in the Credibility Transformer Architecture?\n\n#### Definition of CLS Tokens\nCLS (Class) tokens play a crucial role in the Credibility Transformer architecture. These tokens are scalar values (\\( c_j \\in \\mathbb{R} \\)) that encode one column of the input tensor, forming part of an augmented input tensor denoted as \\( x[+]1:T + 1 \\). Specifically, CLS tokens are represented by \\( c = [c_1,..., c_{2b}]^T \\), where each scalar \\( c_j \\) corresponds to a one-dimensional projection of the corresponding vector in the input tensor. In this augmented input tensor, these CLS tokens occupy the (T + 1)-st component.\n\n#### Usage in Credibility Transformer Architecture\n- **Augmented Input Tensor**: The CLS tokens are part of an augmented input tensor that includes both covariates and positional encodings.\n  \n- **Normalization Layer**: Before entering the Transfor

In [35]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3080 Ti. Max memory = 11.753 GB.
3.275 GB of memory reserved.


## Train

In [36]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,068 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 2,670
 "-____-"     Number of trainable parameters = 194,510,848


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


  0%|          | 1/2670 [00:02<1:32:07,  2.07s/it]

{'loss': 1.8814, 'grad_norm': 1.4999339580535889, 'learning_rate': 4e-05, 'epoch': 0.0}


  0%|          | 2/2670 [00:02<1:02:08,  1.40s/it]

...

{'loss': 0.0043, 'grad_norm': 0.04822973906993866, 'learning_rate': 7.5046904315197e-08, 'epoch': 10.0}


100%|██████████| 2670/2670 [1:05:06<00:00,  1.62s/it]

{'loss': 0.0022, 'grad_norm': 0.034792423248291016, 'learning_rate': 0.0, 'epoch': 10.0}


100%|██████████| 2670/2670 [1:05:07<00:00,  1.46s/it]

{'train_runtime': 3907.5504, 'train_samples_per_second': 2.733, 'train_steps_per_second': 0.683, 'train_loss': 0.18064196587035306, 'epoch': 10.0}


In [22]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2815.5718 seconds used for training.
46.93 minutes used for training.
Peak reserved memory = 5.35 GB.
Peak reserved memory for training = 2.075 GB.
Peak reserved memory % of max memory = 45.52 %.
Peak reserved memory for training % of max memory = 17.655 %.


In [37]:
import pandas as pd
import matplotlib.pyplot as plt
training_df = pd.DataFrame(trainer.state.log_history)

In [38]:
training_df.head(2)

,loss,grad_norm,learning_rate,epoch,step,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,1.8814,1.499934,0.00004,0.003745,1,NaN,NaN,NaN,NaN,NaN
1,1.9541,4.335380,0.00008,0.007491,2,NaN,NaN,NaN,NaN,NaN


In [39]:
from helpers import create_training_plots

In [26]:
fig = create_training_plots(training_df)
fig.show()

In [41]:
training_df.to_csv(f"training_logs/{OUTPUT_MODEL_NAME}.csv", index = False)

## Run Inference

In [42]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def get_response(user_query):
    messages = [
    {"role": "user", "content": user_query},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                            temperature = 1.5, min_p = 0.1)
    return tokenizer.batch_decode(outputs)

In [43]:
dataset_finetune['question'][0]

'Where are the CLS tokens added to the augmented input tensor x[+]1:T +1, and what is their role before and after processing through the Transformer?'

Need to investigate how changing the question affects responses

In [44]:
resp = get_response(dataset_finetune['question'][0])
print(resp[0].split("<|start_header_id|>assistant<|end_header_id|>")[1])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.




The CLS tokens are added as an additional component (the \((T+1)\)-st component) to the augmented input tensor \(x^{+}_{1:T+1}\), which has a shape of \([T + 1] \times 2b\). Specifically, the augmented input tensor is


## Save to HF

In [45]:
print(f"Model dtype: {next(model.parameters()).dtype}")


Model dtype: torch.bfloat16


In [ ]:
model.push_to_hub_gguf(
        f"CPSC532/{OUTPUT_MODEL_NAME}",
        tokenizer,
        # quantization_method = ["q4_k_m", "q8_0", "q5_k_m"], 
        quantization_method = ["not_quantized"], # save original precision (bfloat16)
        token = HF_TOKEN
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 39.33 out of 62.67 RAM for saving.


100%|██████████| 28/28 [00:00<00:00, 87.94it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['bf16'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at CPSC532/2024NOV6_qwen_2_5_model into bf16 GGUF format.
The output location will be /home/owen/Desktop/github/532/implementation/finetuning/CPSC532/2024NOV6_qwen_2_5_model/unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: 2024NOV6_qwen_2_5_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: l

unsloth.BF16.gguf: 100%|██████████| 6.43G/6.43G [04:24<00:00, 24.3MB/s]


Saved GGUF to https://huggingface.co/CPSC532/2024NOV6_qwen_2_5_model


No files have been modified since last commit. Skipping to prevent empty commit.


Saved Ollama Modelfile to https://huggingface.co/CPSC532/2024NOV6_qwen_2_5_model
